# Tutorial 4 — Hybrid Search (Dense + Keyword)

## Where You Are in the Learning Journey

```
 Tutorial 1 --> Tutorial 2 --> Tutorial 3 --> Tutorial 4 --> Tutorial 5
 Basic RAG      Semantic       Reranking      YOU ARE        Benchmark
 (baseline)     Chunking       (T3)           HERE           All Four
```

**What changed from Tutorial 3:** a second retrieval signal — BM25 keyword search —
is added and fused with dense retrieval via Reciprocal Rank Fusion (RRF).

**What you will learn in this tutorial:**
- What BM25 is and how it scores documents using term frequency
- Why keyword retrieval catches things that dense retrieval misses
- What Reciprocal Rank Fusion (RRF) is and why it combines two ranked lists well
- When hybrid search helps and when it does not

```mermaid
flowchart LR
    Q[Query] --> D[Dense Retriever]
    Q --> K[Keyword Retriever BM25]
    D --> F[RRF Fusion]
    K --> F
    F --> L[Top-k for Generation]
```


## Learning Checkpoint: Hybrid Tradeoffs and Readiness for Benchmarking

### What Works Better in Tutorial 4

- **Keyword retrieval** improves exact-token recall (e.g. searching for 'Form A-12'
  or a specific regulation code finds the exact chunk even when embeddings are weak).
- **Dense retrieval** still contributes semantic context (synonyms, paraphrases).
- **Fusion** provides more robust retrieval across different question styles.

### What Still Does Not Work Well

- More moving parts increase tuning complexity (BM25 weights, top-k per source,
  fusion constant k).
- Latency and system complexity are higher than Tutorial 1.
- Hybrid is better in *most* cases but not all — Tutorial 5 shows the data.

### Why Move to Tutorial 5?

We now have four variants.  Tutorial 5 runs all four under identical conditions
and compares them on the same metrics.  That is how we decide which architecture
to deploy for a real system.


In [ ]:
# 1-5) Setup, handbook text, semantic chunks, embeddings, index

import importlib
import os
from pathlib import Path
import shutil
import subprocess
import sys

import pandas as pd
from dotenv import load_dotenv

if shutil.which("uv") is None:
    print("uv not found. Installing with pip...")
    subprocess.run([sys.executable, "-m", "pip", "install", "uv"], check=True)

cwd = Path.cwd().resolve()
repo_root = next(
    (path for path in [cwd, *cwd.parents] if (path / "pyproject.toml").exists() and (path / "src").exists()),
    cwd,
)
os.chdir(repo_root)
src_path = repo_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

REQUIRED_PACKAGES = ["openai", "chromadb", "numpy", "pandas", "rank_bm25", "sentence_transformers", "dotenv"]
PIP_NAME_MAP = {"rank_bm25": "rank-bm25", "sentence_transformers": "sentence-transformers", "dotenv": "python-dotenv"}

def find_missing(packages: list[str]) -> list[str]:
    importlib.invalidate_caches()
    return [pkg for pkg in packages if importlib.util.find_spec(pkg) is None]

missing = find_missing(REQUIRED_PACKAGES)
if missing:
    print("Missing packages:", missing)
    print("Running: uv sync")
    subprocess.run(["uv", "sync"], check=True)

missing_after_sync = find_missing(REQUIRED_PACKAGES)
if missing_after_sync:
    pip_targets = [PIP_NAME_MAP.get(pkg, pkg) for pkg in missing_after_sync]
    print("Installing into current kernel with pip:", pip_targets)
    subprocess.run([sys.executable, "-m", "pip", "install", *pip_targets], check=True)

final_missing = find_missing(REQUIRED_PACKAGES)
if final_missing:
    raise ImportError(f"Dependencies still missing in current kernel: {final_missing}")

from rag_tutorials.io_utils import load_handbook_documents, load_queries
from rag_tutorials.chunking import semantic_chunk_documents
from rag_tutorials.pipeline import build_dense_retriever, build_hybrid_retriever
from rag_tutorials.retrieval import build_bm25, bm25_search
from rag_tutorials.qa import answer_with_context
from rag_tutorials.evaluation import evaluate_single, summarize

load_dotenv()
if not os.getenv("OPENAI_API_KEY"):
    raise EnvironmentError("OPENAI_API_KEY is required")

embedding_model = os.getenv("OPENAI_EMBEDDING_MODEL", "text-embedding-3-small")
chat_model = os.getenv("OPENAI_CHAT_MODEL", "gpt-4.1-mini")

handbook_path = Path("data/handbook_manual.txt")
queries_path = Path("data/queries.jsonl")
if not handbook_path.exists() or not queries_path.exists():
    raise FileNotFoundError("Run: uv run python scripts/generate_data.py")

documents = load_handbook_documents(handbook_path)
queries = load_queries(queries_path)
chunks = semantic_chunk_documents(documents)

dense_retriever, _ = build_dense_retriever(
    chunks=chunks,
    collection_name="tutorial4_dense",
    embedding_model=embedding_model,
)
hybrid_retriever = build_hybrid_retriever(chunks, dense_retriever)

bm25_index, corpus, chunk_ids = build_bm25(chunks)

In [ ]:
# Chunk boundary visualization (same source text, different split strategies)

from rag_tutorials.chunking import fixed_chunk_documents

section_doc = next(doc for doc in documents if doc.section == "International Work")
fixed_view = [c.text for c in fixed_chunk_documents([section_doc], chunk_size=120)]
semantic_view = [c.text for c in semantic_chunk_documents([section_doc])]

print("Section:", section_doc.section)
print("\nFixed chunks:")
for idx, chunk_text in enumerate(fixed_view, start=1):
    print(f"[{idx}] {chunk_text}")

print("\nSemantic chunks:")
for idx, chunk_text in enumerate(semantic_view, start=1):
    print(f"[{idx}] {chunk_text}")

### What Is BM25 and How Does It Score Documents?

**BM25** stands for *Best Match 25*.  It is a keyword-based ranking algorithm
that has been the standard in search engines for decades.
Unlike dense retrieval (which uses vector similarity), BM25 scores documents
purely based on which words they share with the query.

#### The Core Idea: Term Frequency and Document Frequency

BM25 cares about two things:

1. **Term Frequency (TF)** — How many times does the query word appear in this chunk?
   More occurrences = higher score, but with diminishing returns (a word appearing
   10 times is not 10x better than once).

2. **Inverse Document Frequency (IDF)** — How rare is this word across *all* chunks?
   Rare words (e.g. 'repatriation') are more informative than common words (e.g. 'the').
   A rare word appearing in a chunk is a stronger signal of relevance.

```
Query: "Form A-12 reimbursement"

Chunk A: "Submit Form A-12 to request reimbursement for relocation costs."
  -> "Form" appears 1x   IDF: medium  (fairly common word)
  -> "A-12" appears 1x   IDF: HIGH    (very rare — almost unique)
  -> "reimbursement" appears 1x  IDF: medium
  -> BM25 score: HIGH  (rare term 'A-12' matches exactly)

Chunk B: "Employees may request reimbursement for approved expenses."
  -> "Form" appears 0x   (no match for this term)
  -> "A-12" appears 0x   (no match)
  -> "reimbursement" appears 1x
  -> BM25 score: LOW  (only one of three query terms matched)
```

#### Dense vs BM25 — What Each Is Good At

| Scenario | Dense (embedding) | BM25 (keyword) |
|----------|-------------------|----------------|
| Query uses different words than the document | Good (understands synonyms) | Poor (needs exact match) |
| Query contains a specific code or form number | Poor (code has no semantic meaning) | Good (exact token match) |
| Query is conceptual ('what is the leave policy') | Good | Medium |
| Query is precise ('Form A-12 deadline') | Medium | Good |

#### What Is Reciprocal Rank Fusion (RRF)?

After dense and BM25 each produce a ranked list, we need to combine them into
one final list.  **RRF** does this without needing to calibrate the raw scores
(BM25 scores and cosine similarity are on completely different scales).

RRF uses only the **rank position**, not the raw score:

```
RRF score for a chunk = 1 / (k + rank_in_dense_list)
                      + 1 / (k + rank_in_bm25_list)

where k = 60 (a constant that dampens the effect of very high ranks)

Example (k=60):
  Chunk A: rank 1 in dense, rank 1 in BM25
    RRF = 1/(60+1) + 1/(60+1) = 0.0164 + 0.0164 = 0.0328  <- top score

  Chunk B: rank 1 in dense only, not in BM25 top-5
    RRF = 1/(60+1) + 0         = 0.0164 + 0     = 0.0164

  Chunk C: rank 3 in dense, rank 2 in BM25
    RRF = 1/(60+3) + 1/(60+2) = 0.0159 + 0.0161 = 0.0320
```

The chunk that ranks well in *both* lists rises to the top.  A chunk that only one
system finds is not penalised — it still contributes its signal.

The constant k=60 prevents rank 1 from dominating too heavily.  Without it,
rank 1 (score=1.0) would be 10x rank 10 (score=0.1), which is too aggressive.
With k=60, rank 1 (1/61=0.0164) is only ~1.1x rank 10 (1/70=0.0143).


### Two Retrieval Signals: Cosine Similarity (dense) and BM25 (keyword)

**Dense retrieval — cosine similarity (Tutorial 1 baseline)**  
Query and chunk texts are converted to embedding vectors; chunks are ranked by cosine similarity.
Captures *semantic* similarity — works well even when the exact words differ.

**BM25 keyword retrieval (new in this tutorial)**  
Ranks chunks by term-frequency statistics.  
Score ∝ how often query terms appear in the chunk, normalized by chunk length.  
Captures *lexical* similarity — reliable for exact identifiers like `Form A-12`.

**Reciprocal Rank Fusion (RRF)**  
Both ranked lists are merged: `score = 1/(k + rank_dense) + 1/(k + rank_bm25)`.  
Chunks that rank well in *both* lists receive the highest combined score.

See **Tutorial 1, cells 8–10** for the cosine similarity step-by-step walkthrough.
#### Side-by-side toy example: how each signal produces its own top-k

Both dense and keyword retrieval independently rank all chunks and return a top-k list.
Reciprocal Rank Fusion (RRF) then combines the two ranked lists into a single one.

```
Query: 'Form A-12 reimbursement'

Dense retrieval (cosine similarity)   BM25 keyword retrieval
──────────────────────────────────   ──────────────────────
rank 1 [0.88]  expense policy         rank 1 [9.2]  Form A-12 instructions   ← exact match!
rank 2 [0.81]  reimbursement guide    rank 2 [7.8]  reimbursement form guide
rank 3 [0.74]  travel budget rules    rank 3 [5.1]  expense policy
rank 4 [0.68]  form submission guide  rank 4 [3.4]  form submission guide
rank 5 [0.61]  HR contact info        rank 5 [1.9]  travel budget rules

Dense misses 'Form A-12 instructions' at rank 1 — the exact form name doesn't
appear in its training data, so the embedding is not as close as BM25's match.

RRF fusion (k=60 constant):  score = 1/(60+rank_dense) + 1/(60+rank_bm25)

 chunk                        RRF score  final rank
 ──────────────────────       ─────────  ──────────
 Form A-12 instructions       0.01613    1   ← keyword signal promoted it
 expense policy               0.01590    2
 reimbursement form guide     0.01573    3
 form submission guide        0.01524    4
 travel budget rules          0.01506    5
```

BM25's strong exact-match signal for 'Form A-12' lifts that chunk to the top even
though dense retrieval ranked it only 1st — both happen to agree here, but in
cases where they disagree (one scores high, the other low) RRF balances them.

See **Tutorial 1 cells 10–13** for the nearest-neighbor + cosine similarity walkthrough.


In [ ]:
# 6) Implement retrieval functions and inspect dense vs keyword vs hybrid

def dense_only(question: str, top_k: int = 5):
    return dense_retriever(question, top_k=top_k)

def keyword_only(question: str, top_k: int = 5):
    return bm25_search(bm25_index, question, corpus, chunk_ids, top_k=top_k)

def hybrid(question: str, top_k: int = 5):
    return hybrid_retriever(question, top_k=top_k)

probe = "Do I need Form A-12 for my trip?"

dense_df = pd.DataFrame([{"rank": i + 1, "chunk_id": r.chunk_id, "score": r.score, "preview": r.text[:90]} for i, r in enumerate(dense_only(probe))])
keyword_df = pd.DataFrame([{"rank": i + 1, "chunk_id": r.chunk_id, "score": r.score, "preview": r.text[:90]} for i, r in enumerate(keyword_only(probe))])
hybrid_df = pd.DataFrame([{"rank": i + 1, "chunk_id": r.chunk_id, "score": r.score, "preview": r.text[:90]} for i, r in enumerate(hybrid(probe))])

print("Dense only")
display(dense_df)
print("Keyword only")
display(keyword_df)
print("Hybrid")
display(hybrid_df)

In [ ]:
# 7-8) Prompt assembly + end-to-end query

def rag_answer_hybrid(question: str, top_k: int = 5):
    retrieved = hybrid(question, top_k=top_k)
    context = [r.text for r in retrieved]
    answer = answer_with_context(question, context, model=chat_model)
    return answer, retrieved

answer, retrieved = rag_answer_hybrid(probe)
print(answer)

In [ ]:
# 9-10) Evaluation + continuity summary table

rows = [
    evaluate_single(
        query=q,
        retrieval_fn=lambda question: hybrid(question, top_k=5),
        answer_fn=lambda question, context: answer_with_context(question, context, model=chat_model),
        top_k=5,
    )
    for q in queries[:20]
]

print("Tutorial 4 metrics:", summarize(rows))

continuity = pd.DataFrame(
    [
        {"tutorial": 1, "change": "dense baseline with fixed chunks"},
        {"tutorial": 2, "change": "semantic chunking"},
        {"tutorial": 3, "change": "reranking"},
        {"tutorial": 4, "change": "hybrid dense + keyword"},
    ]
)
continuity